# **Import Library**


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from ydata_profiling import ProfileReport

# **Load Data**

In [4]:
train_PATH = 'D:\DAC-2023\Dataset\DataTrain_Preliminary.csv'
pred_PATH = 'D:\DAC-2023\Dataset\Data_Prediction.csv'

Kita memiliki 2 dataset, training dataset dan validation dataset. training dataset akan digunakann untuk melatih model machine learning dan validation dataset akan digunakan untuk mengevaluasi model dan mengukur akurasinya. 

In [15]:
df_t = pd.read_csv(train_PATH, delimiter=';')
df_t.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,type_of_attack
0,0,tcp,private,SH,0,0,0,0,0,0,...,1,0.01,0.94,0.95,0,0.95,1,0,0,nmap
1,0,tcp,private,S0,0,0,0,0,0,0,...,5,0.02,0.08,0.00,0,1.00,1,0,0,neptune
2,0,tcp,http,SF,285,3623,0,0,0,0,...,228,1.00,0.00,0.01,*,0.00,0,0,0,normal
3,0,tcp,http,SF,232,584,0,0,0,0,...,255,1.00,0.00,0.17,*,0.00,0,0,0,normal
4,1,tcp,smtp,SF,1080,327,0,0,0,0,...,154,0.58,0.02,0.00,*,0.00,0,0,0,normal


In [16]:
df_t.shape

(112446, 42)

In [17]:
df_p = pd.read_csv(pred_PATH, delimiter=';')
df_p.head()

,id,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
0,1,1,tcp,smtp,SF,2397,328,0,0,0,...,179,182,0.68,0.02,0.01,0.01,0.0,0.0,0.0,0.0
1,2,990,udp,other,SF,145,105,0,0,0,...,255,1,0.00,0.64,0.94,0.00,0.0,0.0,0.0,0.0
2,3,0,icmp,ecr_i,SF,1032,0,0,0,0,...,255,23,0.09,0.02,0.09,0.00,0.0,0.0,0.0,0.0
3,4,0,icmp,eco_i,SF,8,0,0,0,0,...,1,85,1.00,0.00,1.00,0.51,0.0,0.0,0.0,0.0
4,5,0,udp,domain_u,SF,44,132,0,0,0,...,250,101,0.40,0.01,0.38,0.00,0.0,0.0,0.0,0.0


In [8]:
train_cols = set(df_t.columns)
pred_cols = set(df_p.columns)

print("Columns in df_train but not in df_pred:", train_cols - pred_cols)
print("Columns in df_pred but not in df_train:", pred_cols - train_cols)

Columns in df_train but not in df_pred: {'type_of_attack'}
Columns in df_pred but not in df_train: {'id'}


Saat ini kita fokus terhadap training dataset terlebih dahulu. Pada training data kita memiliki kolom type_of_attack. Untuk menentukan model prediksi yang akan digunakan, maka kita akan menganalisis kolom type_of_attack ini.

# EDA Ala-ala

### Statistika umum

In [9]:
df_train.describe()

,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_serror_rate
count,112118.000000,112203.000000,112008.000000,112018.000000
mean,514.254458,238.042508,53.712086,145.797601
std,7149.084974,4872.291724,2313.831459,3811.799426
min,0.000000,0.000000,0.000000,0.000000
25%,0.050000,0.000000,0.000000,0.000000
50%,0.510000,0.030000,0.000000,0.000000
75%,1.000000,0.070000,0.060000,1.000000
max,99999.000000,99999.000000,99999.000000,99999.000000


### Fitur terkait keamanan

In [24]:
k_secure = df_t[['num_failed_logins', 'num_access_files', 'num_compromised', 'root_shell', 'su_attempted']]
k_secure.head()

,num_failed_logins,num_access_files,num_compromised,root_shell,su_attempted
0,0,0,0,0,0
1,0,0,0,0,*
2,0,0,0,0,0
3,0,0,0,0,0
4,0,0,0,0,0


In [36]:
k_secure.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112446 entries, 0 to 112445
Data columns (total 5 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   num_failed_logins  112446 non-null  object
 1   num_access_files   112446 non-null  object
 2   num_compromised    112446 non-null  object
 3   root_shell         112446 non-null  object
 4   su_attempted       112446 non-null  object
dtypes: object(5)
memory usage: 4.3+ MB


In [27]:
k_secure_sorted1 = k_secure.sort_values(by='num_failed_logins', ascending=False)
k_secure_sorted1.head()

,num_failed_logins,num_access_files,num_compromised,root_shell,su_attempted
110338,99999,0,0,0,0
45932,99999,0,0,0,0
30276,99999,0,0,0,0
85830,99999,0,0,0,0
110455,99999,0,0,0,0


In [28]:
k_secure_sorted2 = k_secure.sort_values(by='num_access_files', ascending=False)
k_secure_sorted2.head()

,num_failed_logins,num_access_files,num_compromised,root_shell,su_attempted
100990,0,99999,0,0,0
46499,0,99999,0,0,0
35873,0,99999,0,0,0
60012,0,99999,0,0,0
64112,0,99999,0,0,0


In [29]:
k_secure_sorted3 = k_secure.sort_values(by='num_compromised', ascending=False)
k_secure_sorted3.head()

,num_failed_logins,num_access_files,num_compromised,root_shell,su_attempted
79805,0,0,99999,0,0
36650,0,0,99999,0,0
33235,0,0,99999,0,0
90245,0,0,99999,0,0
72852,0,0,99999,0,0


In [30]:
k_secure_sorted4 = k_secure.sort_values(by='root_shell', ascending=False)
k_secure_sorted4.head()

,num_failed_logins,num_access_files,num_compromised,root_shell,su_attempted
107786,0,0,0,99999,0
64189,0,0,0,99999,0
72672,0,0,0,99999,0
104207,0,0,0,99999,0
58819,0,0,0,99999,0


In [31]:
k_secure_sorted5 = k_secure.sort_values(by='su_attempted', ascending=False)
k_secure_sorted5.head()

,num_failed_logins,num_access_files,num_compromised,root_shell,su_attempted
29318,0,0,0,0,99999
77621,0,0,0,0,99999
95250,0,0,0,0,99999
102248,0,0,0,0,99999
2544,0,0,0,0,99999


In [32]:
k_secure.describe()

,num_failed_logins,num_access_files,num_compromised,root_shell,su_attempted
count,112446,112446,112446,112446,112446
unique,7,12,82,4,5
top,0,0,0,0,0
freq,111800,111500,110875,112009,112100


In [ ]:
k_secure['num_failed_logins'].value_counts()

### Terkait koneksi

In [33]:
c_conection = df_t[['duration', 'protocol_type', 'service', 'flag', 'src_bytes', 'dst_bytes']]
c_conection.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes
0,0,tcp,private,SH,0,0
1,0,tcp,private,S0,0,0
2,0,tcp,http,SF,285,3623
3,0,tcp,http,SF,232,584
4,1,tcp,smtp,SF,1080,327


In [34]:
c_conection.describe()

,duration,protocol_type,service,flag,src_bytes,dst_bytes
count,112446,111778,111426,111593,112446,112446
unique,2665,3,70,11,3201,8843
top,0,tcp,http,SF,0,0
freq,102916,90997,36013,65941,44030,60138


In [35]:
c_conection.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112446 entries, 0 to 112445
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   duration       112446 non-null  object
 1   protocol_type  111778 non-null  object
 2   service        111426 non-null  object
 3   flag           111593 non-null  object
 4   src_bytes      112446 non-null  object
 5   dst_bytes      112446 non-null  object
dtypes: object(6)
memory usage: 5.1+ MB


## Kategori Variabel

In [14]:
k_vars = df_train[['protocol_type', 'service', 'flag']]
k_vars.head()

,protocol_type,service,flag
0,tcp,private,SH
1,tcp,private,S0
2,tcp,http,SF
3,tcp,http,SF
4,tcp,smtp,SF


## **Looking at Type of Attack**

label column : type of attack

In [17]:
attack_type = df_train['type_of_attack'].unique()

print("Nilai yang ada dalam kolom x3:")
print(attack_type)

Nilai yang ada dalam kolom x3:
['nmap' 'neptune' 'normal' 'Denial of Service Attack' 'portsweep' 'satan'
 'ipsweep' 'smurf']


TYPE OF ATTACK :
- nmap
- neptune
- normal
- Denial of Service Attack
- portsweep
- satan
- ipsweep
- smurf

Normal

DOS ATTACK: 
neptune -> working to SYN flood Denial of Service on one or more ports, Neptune attack make the memory resources busy by sending TCP packet requesting to initiate a TCP session,
DOS ->  makes the computer system to be hectic by sending many ping, 
Smurf -> Denial of Service ICMP echo reply flood. 

PROBE: nmap, portsweep, satan, ipsweep, 

This paper focused on Neptune and Smurf types of attack. Those attack characterized as DoS Attack. Dos Attack makes the computer system to be hectic by sending many ping. Neptune attack working to SYN flood Denial of Service on one or more ports while the function of Smurf attack is Denial of Service ICMP echo reply flood. Neptune attack make the memory resources busy by sending TCP packet requesting to initiate a TCP session.  After many TCP packet sent, the computer system finally runs out of memory resources. Smurf attack is the popular type of attack sending ICMP echo request packet to middle device. ICMP packets have source address name as victim’s IP address and middle device address as destination address. DoS attack reduces the performance of server by overflowing the ICMP traffic as discussed above in the Neptune and Smurf attack. So, detection of DoS attack is very significant to protect the system. 

KOLOM KUNCI:
- protocol type
- port
- 

In [35]:
def filtered_by_type_of_attack(df, values):
    filtered = df[df['type_of_attack'] == values]
    return filtered

In [48]:
filtered_by_type_of_attack(df_train, 'nmap')

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,type_of_attack
0,0,tcp,private,SH,0,0,0,0,0,0,...,1,0.01,0.94,0.95,0,0.95,1,0,0,nmap
41,0,icmp,eco_i,SF,8,0,0,0,0,0,...,83,1.00,0.00,1.00,*,0.00,0,0,0,nmap
73,0,tcp,private,SH,0,0,0,0,0,0,...,1,0.01,0.89,0.91,0,0.91,1,0,0,nmap
159,0,icmp,eco_i,SF,8,0,0,0,0,0,...,34,1.00,0.00,1.00,*,0.00,0,0,0,nmap
208,0,tcp,private,SH,0,0,0,0,0,0,...,1,0.03,0.73,0.76,0,0.76,1,0,0,nmap
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111898,0,tcp,ctf,SH,0,0,0,0,0,0,...,1,0.01,0.94,0.95,0,0.95,1,0,0,nmap
111899,0,icmp,eco_i,SF,8,0,0,0,0,0,...,143,1.00,0.00,1.00,*,0.00,0,0,0,nmap
112060,0,udp,private,SF,207,0,0,0,0,0,...,108,0.87,0.02,0.87,0,0.00,0,0,0,nmap
112394,0,udp,private,SF,207,0,0,0,0,0,...,100,0.86,0.02,0.86,0,0.00,0,0,0,nmap


In [ ]:
ProfileReport(filtered_by_type_of_attack(df_train, 'nmap'))

In [39]:
filtered_by_type_of_attack(df_train, 'neptune')

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,type_of_attack
1,0,tcp,private,S0,0,0,0,0,0,0,...,5,0.02,0.08,0.0,0,1.0,1,0,0,neptune
5,0,tcp,discard,S0,0,0,0,0,0,0,...,18,0.07,0.05,0.0,0,1.0,1,0,0,neptune
7,0,tcp,exec,S0,0,0,0,0,0,0,...,25,0.10,0.05,0.0,0,1.0,1,0,0,neptune
12,0,tcp,private,REJ,0,0,0,0,0,0,...,6,0.02,0.07,0.0,0,0.0,0,1,1,neptune
13,0,tcp,uucp_path,S0,0,0,0,0,0,0,...,11,0.04,0.08,0.0,0,1.0,1,0,0,neptune
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112431,0,tcp,private,S0,0,0,0,0,0,0,...,4,0.02,0.06,0.0,0,1.0,1,0,0,neptune
112435,0,tcp,klogin,S0,0,0,0,0,0,0,...,4,0.02,0.05,0.0,0,1.0,1,0,0,neptune
112436,0,tcp,auth,S0,0,0,0,0,0,*,...,6,0.02,0.07,0.0,0,1.0,1,0,0,neptune
112439,0,tcp,ssh,S0,0,0,0,0,0,0,...,3,0.01,0.09,0.0,0,1.0,1,0,0,neptune


In [40]:
filtered_by_type_of_attack(df_train, 'normal')

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,type_of_attack
2,0,tcp,http,SF,285,3623,0,0,0,0,...,228,1.00,0.00,0.01,*,0.00,0,0,0,normal
3,0,tcp,http,SF,232,584,0,0,0,0,...,255,1.00,0.00,0.17,*,0.00,0,0,0,normal
4,1,tcp,smtp,SF,1080,327,0,0,0,0,...,154,0.58,0.02,0.00,*,0.00,0,0,0,normal
6,0,udp,domain_u,SF,43,43,0,0,0,0,...,254,1.00,0.01,0.01,0,0.00,0,0,0,normal
8,0,tcp,http,SF,284,614,0,0,0,0,...,255,1.00,0.00,0.10,*,0.00,0,0,0,normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112440,5,tcp,smtp,SF,1377,328,0,0,0,0,...,117,0.66,0.03,0.01,0,0.00,0,0,0,normal
112441,0,tcp,http,REJ,0,0,0,0,0,0,...,255,1.00,0.00,0.14,*,0.00,0,*,*,normal
112442,0,tcp,http,SF,309,758,0,0,0,0,...,255,1.00,0.00,0.02,*,0.02,*,0,0,normal
112443,0,tcp,http,SF,363,721,0,0,0,0,...,255,1.00,0.00,0.00,0,0.00,0,0,0,normal


In [41]:
filtered_by_type_of_attack(df_train, 'Denial of Service Attack')

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,type_of_attack
10,0,tcp,http,SF,54540,8314,0,0,0,2,...,182,1.00,0.00,0.01,0,0.00,0,*,*,Denial of Service Attack
14,0,udp,private,SF,28,0,0,3,0,0,...,9,0.04,0.02,0.04,0,0.00,0,*,0,Denial of Service Attack
56,9,tcp,http,SF,54540,8314,0,0,0,2,...,24,1.00,0.00,0.04,0,0.00,0,*,*,Denial of Service Attack
91,1,tcp,http,SF,54540,8314,0,0,0,2,...,255,1.00,0.00,0.00,0,0.00,0,*,*,Denial of Service Attack
258,0,icmp,ecr_i,SF,1480,0,0,1,0,0,...,19,0.50,1.00,0.50,*,0.00,0,0,0,Denial of Service Attack
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112276,0,tcp,http,SF,54540,8314,0,0,0,2,...,20,1.00,0.00,0.05,0,0.00,0,*,*,Denial of Service Attack
112284,0,udp,private,SF,28,0,0,3,0,0,...,37,0.15,0.01,0.15,0,0.00,0,0,0,Denial of Service Attack
112355,0,icmp,ecr_i,SF,1480,0,0,1,0,*,...,15,0.15,0.04,0.15,0,0.00,0,0,0,Denial of Service Attack
112403,0,icmp,NaN,SF,1480,0,0,*,0,0,...,39,1.00,0.00,1.00,*,0.00,0,0,0,Denial of Service Attack


In [42]:
filtered_by_type_of_attack(df_train, 'portsweep')

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,type_of_attack
20,0,tcp,private,RSTR,0,0,0,0,0,0,...,1,0.00,0.34,0.33,0,0.0,0,*,1,portsweep
74,41782,tcp,private,RSTR,1,0,0,0,0,0,...,2,0.01,0.23,0.45,0,0.0,0,*,1,portsweep
79,31481,tcp,private,RSTR,1,0,0,0,0,0,...,2,0.01,0.82,1.00,0,0.0,0,1,1,portsweep
117,40232,tcp,private,RSTR,1,0,0,0,0,0,...,2,0.01,0.45,0.90,0,0.0,0,*,1,portsweep
172,0,tcp,mtp,RSTR,0,0,0,0,0,0,...,1,0.00,0.25,0.48,0,0.0,0,*,1,portsweep
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112217,38933,tcp,private,RSTR,1,0,0,0,0,0,...,2,0.01,0.51,1.00,0,0.0,0,1,1,portsweep
112251,4,tcp,private,RSTR,0,0,0,0,0,0,...,2,0.01,0.26,0.49,0,0.0,0,*,1,portsweep
112339,4,tcp,other,RSTR,0,0,0,0,0,0,...,1,0.00,0.62,1.00,0,0.0,0,1,1,portsweep
112414,0,tcp,private,RSTR,*,0,0,0,0,0,...,1,0.01,0.15,0.12,0,0.0,0,*,1,portsweep


In [43]:
filtered_by_type_of_attack(df_train, 'satan')

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,type_of_attack
52,0,tcp,other,S0,0,0,0,0,0,0,...,1,0.00,1.00,0.00,0,0.14,1,*,0,satan
83,0,udp,private,SF,1,0,0,0,0,0,...,204,0.80,0.03,1.00,0,0.00,0,0,0,satan
100,0,udp,private,SF,1,0,0,0,0,0,...,23,0.09,0.17,1.00,0,0.00,0,0,0,satan
127,0,tcp,private,REJ,0,0,0,0,0,0,...,1,0.00,0.70,0.00,0,0.03,0,*,1,satan
146,0,tcp,private,REJ,0,*,0,0,0,0,...,1,0.00,1.00,0.00,0,0.13,0,*,1,satan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112307,0,udp,private,SF,1,0,0,0,0,0,...,31,0.12,0.25,0.74,0,0.00,0,*,0,satan
112320,0,tcp,other,REJ,0,0,0,0,0,0,...,1,0.00,1.00,0.00,0,0.24,0,*,1,satan
112383,0,tcp,other,REJ,0,0,0,0,0,0,...,1,0.00,1.00,0.00,0,0.10,0,*,1,satan
112402,1,tcp,finger,SF,6,32,0,0,0,0,...,3,0.01,0.02,0.00,0,0.00,0,0,0,satan


In [44]:
filtered_by_type_of_attack(df_train, 'ipsweep')

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,type_of_attack
58,0,tcp,private,REJ,0,0,0,0,0,0,...,2,0.03,1.0,0.03,1,0.0,0,*,1,ipsweep
76,0,icmp,eco_i,SF,8,0,0,0,0,0,...,119,1.00,0.0,1.00,*,0.0,0,0,0,ipsweep
215,0,icmp,eco_i,SF,18,0,0,0,0,0,...,48,1.00,0.0,1.00,1,0.0,0,0,0,ipsweep
284,0,icmp,eco_i,SF,18,0,0,0,0,0,...,42,1.00,0.0,1.00,1,0.0,0,0,0,ipsweep
337,0,icmp,eco_i,SF,8,0,0,0,0,0,...,15,1.00,0.0,1.00,*,0.0,0,0,0,ipsweep
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112272,0,icmp,eco_i,SF,8,0,0,0,0,0,...,140,1.00,0.0,1.00,*,0.0,0,0,0,ipsweep
112372,0,icmp,eco_i,SF,8,0,0,0,0,0,...,171,1.00,0.0,1.00,*,0.0,0,0,0,ipsweep
112373,0,icmp,eco_i,SF,8,0,0,0,0,0,...,99,1.00,0.0,1.00,*,0.0,0,0,0,ipsweep
112380,0,icmp,eco_i,SF,18,0,0,0,0,0,...,145,1.00,0.0,1.00,1,0.0,0,0,0,ipsweep


In [45]:
filtered_by_type_of_attack(df_train, 'smurf')

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,type_of_attack
61,0,icmp,ecr_i,SF,1032,0,0,0,0,0,...,118,0.46,0.07,0.46,0,0.00,0,0,0,smurf
81,0,icmp,ecr_i,SF,1032,0,0,0,0,0,...,6,0.02,0.26,0.02,0,0.00,0,0,0,smurf
375,0,icmp,ecr_i,SF,1032,0,0,0,0,0,...,200,0.78,0.02,0.78,0,0.00,0,*,0,smurf
419,0,icmp,ecr_i,SF,1032,0,0,0,0,0,...,198,0.78,0.02,0.78,0,0.00,0,0,0,smurf
483,0,icmp,ecr_i,SF,520,0,0,0,0,0,...,255,1.00,0.00,1.00,0,0.00,0,0,0,smurf
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112229,0,icmp,ecr_i,SF,1032,0,0,0,0,0,...,97,0.51,0.04,0.51,0,0.01,0,*,0,smurf
112309,0,icmp,ecr_i,SF,1032,0,0,0,0,0,...,131,0.51,0.01,0.51,0,0.00,0,0,0,smurf
112310,0,icmp,ecr_i,SF,1032,0,0,0,0,0,...,41,0.30,0.05,0.30,0,0.01,0,*,0,smurf
112350,0,icmp,ecr_i,SF,520,0,0,0,0,0,...,255,1.00,0.00,1.00,0,0.00,0,0,0,smurf


# **Duplicate Data**

In [20]:
duplicates = df_train[df_train.duplicated()]

In [21]:
duplicates

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,type_of_attack
2439,0,tcp,other,REJ,0,0,0,0,0,0,...,1,0.00,1.00,0.00,0,0.26,0,*,1,satan
3041,0,tcp,http,REJ,0,0,0,0,0,0,...,255,1.00,0.00,0.50,*,0.00,0,1,*,normal
3411,0,tcp,http,REJ,0,0,0,0,0,0,...,255,1.00,0.00,0.50,*,0.00,0,1,*,normal
4164,0,tcp,other,REJ,0,0,0,0,0,0,...,1,0.00,1.00,0.00,0,0.18,0,*,1,satan
4714,0,icmp,eco_i,SF,8,0,0,0,0,0,...,73,1.00,0.00,1.00,*,0.00,0,0,0,ipsweep
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112304,0,tcp,other,REJ,0,0,0,0,0,0,...,1,0.00,1.00,0.00,0,0.19,0,*,1,satan
112320,0,tcp,other,REJ,0,0,0,0,0,0,...,1,0.00,1.00,0.00,0,0.24,0,*,1,satan
112383,0,tcp,other,REJ,0,0,0,0,0,0,...,1,0.00,1.00,0.00,0,0.10,0,*,1,satan
112389,0,tcp,private,S0,0,0,0,0,0,0,...,23,0.09,0.06,0.00,0,1.00,1,0,0,neptune


is it really duplicate? harus nemu variabel kunci yg menentukan itu row duplicate or no. atau jangan jangan nilai duplicate ini bisa jadi kunci network attack (karena patternnya sama)